### For optimal use of this notebook, the T4 GPU runtime is recommended. ###

# Question 1 #

_Use Transformers model and build a Code Generation application on the given data.

_Data: https://huggingface.co/datasets/theblackcat102/evol-codealpaca-v1

Approach used: Finetuned from model Llama 2-7b.

## Setup ##

In [ ]:

!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q -U datasets bitsandbytes einops wandb
!pip install -q huggingface_hub

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.0/118.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 73.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 7.

## Dataset ##

In [ ]:

from datasets import load_dataset

dataset_name = "theblackcat102/evol-codealpaca-v1"
dataset = load_dataset(dataset_name, split='train')



Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

## Loading the model ##

In [ ]:

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "Trelis/Llama-2-7b-chat-hf-sharded-bf16"

# Quantization Config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

# Model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [ ]:
# let's also load the tokenizer below

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
# Below we will load the configuration file in order to create the LoRA model

from peft import LoraConfig, get_peft_model

# LoRA Config
lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    target_modules=["self_attn.q_proj", "self_attn.k_proj", "self_attn.v_proj", "self_attn.o_proj"],
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

## Loading the trainer ##

In [ ]:

from transformers import TrainingArguments

# Training Params
output_dir = "./results"
per_device_train_batch_size = 1
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 25
logging_steps = 25
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 250
warmup_ratio = 0.03
lr_scheduler_type = "constant"


training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    gradient_checkpointing=True,
)

In [ ]:
# Then finally pass everthing to the trainer

from trl import SFTTrainer

max_seq_length = 1024

# Trainer fine-tuning
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field='instruction',
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:133: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/111272 [00:00<?, ? examples/s]

In [ ]:
# We will also pre-process the model by upcasting the layer norms in float 32 for more stable training

for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

## Training the model ##

In [ ]:
# Now let's train the model! Simply call trainer.train()

trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
25,2.156900
50,2.441300
75,1.778900
100,2.260300
125,1.752100
150,2.268400
175,1.731400
200,2.254300
225,1.675700
250,2.295400


TrainOutput(global_step=250, training_loss=2.0614732971191407, metrics={'train_runtime': 1875.8029, 'train_samples_per_second': 0.533, 'train_steps_per_second': 0.133, 'total_flos': 7024273883381760.0, 'train_loss': 2.0614732971191407, 'epoch': 0.01})

In [ ]:
# The `SFTTrainer` will take care of properly saving only the adapters during training instead of saving the entire model.

model.eval()


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(
            in_features=4096, out_features=4096, bias=False
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.1, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=64, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=64, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (k_proj): Linear4bit(
            in_features=4096, out_features=4096, bias=False
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.1, inplace=False)
            )
            (lora_A): ModuleDict(
  

## Save the model ##

In [ ]:

refined_model = "llama-2-7b-CH1-cav1-3"
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained(refined_model)

## Loading the model finetuned ##

In [ ]:

from peft import LoraConfig, get_peft_model

lora_config = LoraConfig.from_pretrained('llama-2-7b-CH1-cav1-3')
model = get_peft_model(model, lora_config)

### Predictions ###

In [ ]:

from transformers import TextStreamer

device = "cuda:0"
system_prompt = "You are an expert assistant in the Python programming language, who provides precise, correct and concise answers."
limit_prompt ="Only answer if the question is related to the Python programming language. Otherwise reject the question formally."
user_prompt="Answer the following correctly: Write a Python function to check if a given list contains any duplicate elements."

B_INST, E_INST = "[INST]", "[/INST]"
B_LIM, E_LIM = "[LIM>>]", "[<</LIM]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"

prompt = f"{B_INST} {B_SYS}{system_prompt.strip()}{E_SYS}{B_LIM}Limit: {limit_prompt.strip()}{E_LIM}\n\n{user_prompt.strip()} {E_INST}\n\n"

inputs = tokenizer([prompt], return_tensors="pt").to(device)

streamer = TextStreamer(tokenizer)

outputs = model.generate(**inputs, streamer=streamer, max_new_tokens=512)


<s> [INST] <<SYS>>
You are an expert assistant in the Python programming language, who provides precise, correct and concise answers.
<</SYS>>

[LIM>>]Limit: Only answer if the question is related to the Python programming language. Otherwise reject the question formally.[<</LIM]

Answer the following correctly: Write a Python function to check if a given list contains any duplicate elements. [/INST]



/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1421: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


Of course! Here is a Python function that checks if a given list contains any duplicate elements:
```
def are_there_duplicates(my_list):
   return any(x == y for x, y in itertools.combinations(my_list, 2))

Explanation:
The `any()` function is used to check if any of the elements in the list are duplicates. The `itertools.combinations()` function is used to generate all possible pairs of elements in the list, and the `x == y` expression is used to check if two elements are duplicates.
Here is an example of how to use this function:
```
my_list = [1, 2, 3, 4, 5, 5, 6]
if are_there_duplicates(my_list):
   print("There are duplicates in the list!")
else:
   print("There are no duplicates in the list.")

```
I hope this helps! Let me know if you have any questions.</s>


In [ ]:
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

[INST] <<SYS>>
You are an expert assistant in the Python programming language, who provides precise, correct and concise answers.
<</SYS>>

[LIM>>]Limit: Only answer if the question is related to the Python programming language. Otherwise reject the question formally.[<</LIM]

Answer the following correctly: Write a Python function to check if a given list contains any duplicate elements. [/INST]

Of course! Here is a Python function that checks if a given list contains any duplicate elements:
```
def are_there_duplicates(my_list):
    return any(x == y for x, y in itertools.combinations(my_list, 2))

Explanation:
The `any()` function is used to check if any of the elements in the list are duplicates. The `itertools.combinations()` function is used to generate all possible pairs of elements in the list, and the `x == y` expression is used to check if two elements are duplicates.
Here is an example of how to use this function:
```
my_list = [1, 2, 3, 4, 5, 5, 6]
if are_there_duplicates(

In [ ]:
# Response evaluation
import itertools

def are_there_duplicates(my_list):
    return any(x == y for x, y in itertools.combinations(my_list, 2))

my_list = [1,2,3,4,5,5,6]
if are_there_duplicates(my_list):
    print("There are duplicates in the list!")
else:
    print("There are no duplicates in the list.")

There are duplicates in the list!


## Push to Huggingface ##

In [ ]:

from huggingface_hub import notebook_login
notebook_login()



In [ ]:
model.push_to_hub(refined_model, use_auth_token=True, max_shard_size="5GB")
tokenizer.push_to_hub(refined_model, use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:844: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


adapter_model.bin:   0%|          | 0.00/269M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/LeanAI/llama-2-7b-CH1-cav1-3/commit/d8c1d7a1f0ce4a65b2813f06d3fd9a323cece883', commit_message='Upload tokenizer', commit_description='', oid='d8c1d7a1f0ce4a65b2813f06d3fd9a323cece883', pr_url=None, pr_revision=None, pr_num=None)

## Loading the model finetuned from the Hugging Face ##

Link to trained model:

https://huggingface.co/LeanAI/llama-2-7b-CH1-cav1-3

In [ ]:

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "LeanAI/llama-2-7b-CH1-cav1-3"
model = AutoModelForCausalLM.from_pretrained(peft_model_id, device_map="auto", load_in_4bit=True)
tokenizer = AutoTokenizer.from_pretrained(peft_model_id)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Predictions ###

In [ ]:
from transformers import TextStreamer

user_prompt1 = "Answer the following correctly: Develop an example of a function in Python that returns the sum of two numbers."
user_prompt2 = "Answer the following correctly: Write a function in Python to check if a given list contains any duplicate elements."
user_prompt3 = "Answer the following correctly: Write a function in python that returns even numbers between 0 and 20 (including 20)."
user_prompt4 = "What is the capital of France?"

# Creating function assistant_app
def assistant_app(user_prompt):
    device = "cuda:0"
    system_prompt = "You are an expert assistant in the Python programming language, who provides precise, correct and concise answers."
    limit_prompt ="Only answer if the question is related to the Python programming language. Otherwise reject the question formally."

    B_INST, E_INST = "[INST]", "[/INST]"
    B_LIM, E_LIM = "[LIM>>]", "[<</LIM]"
    B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"

    prompt = f"{B_INST} {B_SYS}{system_prompt.strip()}\n{B_LIM}: {limit_prompt.strip()}{E_LIM}{E_SYS}Human: {user_prompt.strip()} {E_INST}\n\nAssistant: "

    inputs = tokenizer([prompt], return_tensors="pt").to(device)

    streamer = TextStreamer(tokenizer)

    # Text generation strategies
    _ = model.generate(**inputs, streamer=streamer, max_new_tokens=512, do_sample=True, temperature=0.9, top_k=11)

In [ ]:
assistant_app(user_prompt1)

<s> [INST] <<SYS>>
You are an expert assistant in the Python programming language, who provides precise, correct and concise answers.
[LIM>>]: Only answer if the question is related to the Python programming language. Otherwise reject the question formally.[<</LIM]
<</SYS>>

Human: Answer the following correctly: Develop an example of a function in Python that returns the sum of two numbers. [/INST]

Assistant:  Of course! Here's an example of a function in Python that takes two arguments, `a` and `b`, and returns their sum:
```
def sum_of_two(a, b):
   return a + b

For example, you could call this function like this:

result = sum_of_two(3, 4)

Which would return the value 7.

Is there anything else I can help you with?
```

Please let me know if you have any other questions!</s>


In [ ]:
# Evaluating the response of the intelligent assistant.
def add_numbers(x,y):
    return x+y

result = add_numbers(3,5)
print(result)

8


In [ ]:
assistant_app(user_prompt2)

<s> [INST] <<SYS>>
You are an expert assistant in the Python programming language, who provides precise, correct and concise answers.
[LIM>>]: Only answer if the question is related to the Python programming language. Otherwise reject the question formally.[<</LIM]
<</SYS>>

Human: Answer the following correctly: Write a function in Python to check if a given list contains any duplicate elements. [/INST]

Assistant:  Of course! Here is a function in Python that checks if a given list contains any duplicate elements:
def has_ duplicates(list):
   # Initialize a set to store the unique elements
   unique = set()
   # Iterate over the list and add each element to the set
   for element in list:
       # If the element is already in the set, return True
       if element in unique:
           return True
   # If the list does not contain any duplicates, return False
   return False

You can use this function by calling it with a list as an argument, like this:
has_duplicates([1, 2, 3, 3, 4

In [ ]:
# Evaluating the response of the intelligent assistant.
def check_duplicates(my_list):

    duplicate_elements = set()
    for element in my_list:
        if element in duplicate_elements:
            return True
        else:
            duplicate_elements.add(element)

    return False

list_1 = [1,2,3,4,5,6,6]

check_duplicates(list_1)


True

In [ ]:
assistant_app(user_prompt3)

<s> [INST] <<SYS>>
You are an expert assistant in the Python programming language, who provides precise, correct and concise answers.
[LIM>>]: Only answer if the question is related to the Python programming language. Otherwise reject the question formally.[<</LIM]
<</SYS>>

Human: Answer the following correctly: Write a function in python that returns even numbers between 0 and 20 (including 20). [/INST]

Assistant:  Of course! I'd be happy to help. To write a function in Python that returns even numbers between 0 and 20 (including 20), you can use the following code:
def even_numbers(lower, upper):
return [x for x in range(lower, upper + 1) if x % 2 == 0]
This function takes two arguments, `lower` and `upper`, which are the lower and upper limits of the range of numbers to be returned. The function then uses a list comprehension to generate all the even numbers within that range.
For example, if you call the function with `lower=0` and `upper=20`, it will return the following list of

In [ ]:
# Evaluating the response of the intelligent assistant.
def even_numbers(lower_bound: int, upper_bound: int) -> list:
    return [x for x in range(lower_bound, upper_bound + 1) if x % 2==0]

#even_numbers(0,20)
print(even_numbers(0,20))

def even_numbers_between(lower, upper):
    even_numbers = []
    for num in range(lower, upper +1):
        if num % 2 == 0:
            even_numbers.append(num)
    return even_numbers

print(even_numbers_between(0,20))

even_num = [num for num in range(0,20) if num % 2==0]
print(even_num)

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20]
[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20]
[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]


In [ ]:
assistant_app(user_prompt4)

<s> [INST] <<SYS>>
You are an expert assistant in the Python programming language, who provides precise, correct and concise answers.
[LIM>>]: Only answer if the question is related to the Python programming language. Otherwise reject the question formally.[<</LIM]
<</SYS>>

Human: What is the capital of France? [/INST]

Assistant:  I'm just an AI and do not have access to information outside of the Python programming language. Therefore, I cannot answer your question about the capital of France. Please refrain from asking questions that are outside of my expertise. Is there anything else I can help you with regarding the Python language?</s>


# Question 2 #

_Use GenAI Stack and build an ChatBot application like ChatGPT on your own data.

_You are free to use any default data type(CSV/PDF/JSON).

_Documentation: https://genaistack.aiplanet.com/

_Since, we have already added Chat with Webpage in Module 14, we will not accept notebook with Webpage for assessment submission. Submission with Webpage as data type will only be accepted, if it has certain application use case.

Approach used: Chat with Your Data using OpenAI API Key.

## Installation ##

In [ ]:

!pip install git+https://github.com/aiplanethub/genai-stack.git

  Cloning https://github.com/aiplanethub/genai-stack.git to /tmp/pip-req-build-v5waxo69
  Running command git clone --filter=blob:none --quiet https://github.com/aiplanethub/genai-stack.git /tmp/pip-req-build-v5waxo69
  Resolved https://github.com/aiplanethub/genai-stack.git to commit a46a22c26558c48ee389da6af8c0199044f1c399
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 866.9/866.9 kB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 60

In [ ]:
!genaistack version


 ██████╗ ███████╗███╗   ██╗ █████╗ ██╗    ███████╗████████╗ █████╗  ██████╗██╗  ██╗
██╔════╝ ██╔════╝████╗  ██║██╔══██╗██║    ██╔════╝╚══██╔══╝██╔══██╗██╔════╝██║ ██╔╝
██║  ███╗█████╗  ██╔██╗ ██║███████║██║    ███████╗   ██║   ███████║██║     █████╔╝
██║   ██║██╔══╝  ██║╚██╗██║██╔══██║██║    ╚════██║   ██║   ██╔══██║██║     ██╔═██╗
╚██████╔╝███████╗██║ ╚████║██║  ██║██║    ███████║   ██║   ██║  ██║╚██████╗██║  ██╗
 ╚═════╝ ╚══════╝╚═╝  ╚═══╝╚═╝  ╚═╝╚═╝    ╚══════╝   ╚═╝   ╚═╝  ╚═╝ ╚═════╝╚═╝  ╚═╝

Version - 0.2.3


## Import requerid modules ##

In [ ]:

import os
from getpass import getpass

from genai_stack.stack.stack import Stack
from genai_stack.etl.langchain import LangchainETL
from genai_stack.embedding.langchain import LangchainEmbedding
from genai_stack.vectordb.chromadb import ChromaDB
from genai_stack.prompt_engine.engine import PromptEngine
from genai_stack.model.gpt3_5 import OpenAIGpt35Model
from genai_stack.retriever.langchain import LangChainRetriever
from genai_stack.memory.langchain import ConversationBufferMemory

## Dataset ##

Data: https://www.kaggle.com/datasets/linkanjarad/coding-problems-and-solution-python-code

Access to data on drive: https://drive.google.com/file/d/1oNIsg1tCc_XsPeG64wNN4oe8WIbQtDQi/view?usp=sharing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

data = "/content/drive/MyDrive/Colab Notebooks/Bootcamp LLM/data/ProblemSolutionPythonV3.csv"

## ETL ##

In [ ]:

etl = LangchainETL.from_kwargs(name="CSVLoader",
                               fields={"file_path": data
                        }
)

## Create Embeddings to store in VectorDB ##

In [ ]:

config = {
    "model_name": "sentence-transformers/all-mpnet-base-v2",
    "model_kwargs": {"device": "cpu"},
    "encode_kwargs": {"normalize_embeddings": False},
}
embedding = LangchainEmbedding.from_kwargs(name="HuggingFaceEmbeddings", fields=config)

## Define the VectorDB ##

In [ ]:

chromadb = ChromaDB.from_kwargs()

## Define your LLM ##

In [ ]:
# Setup your API Key

api_key = getpass("Enter OpenAI API Key:")
os.environ['OPENAI_API_KEY'] = api_key

Enter OpenAI API Key:··········


In [ ]:

llm = OpenAIGpt35Model.from_kwargs(parameters={"openai_api_key": api_key})


## Add retrieval and Stack all the componets ##

In [ ]:

prompt_engine = PromptEngine.from_kwargs(should_validate=False)
retriever = LangChainRetriever.from_kwargs()
memory = ConversationBufferMemory.from_kwargs()
Stack(
    etl=etl,
    embedding=embedding,
    vectordb=chromadb,
    model=llm,
    prompt_engine=prompt_engine,
    retriever=retriever,
    memory=memory
)

## Run your ETL ##

In [ ]:

etl.run()

## Predictions ##

In [ ]:

prompt_1 = "What programming language is used in the document we are working on?"
promtp_2 = "Write a Python program to create Fibonacci series upto n using Lambda."
prompt_3 = "what is python?"

In [ ]:
response = retriever.retrieve(prompt_1)
print(response['output'])

The programming language used in the document we are working on is Python.


In [ ]:
# Creating function genai_app

def genai_app(prompt):
    response = retriever.retrieve(prompt)
    print(response['output'])

In [ ]:
genai_app(promtp_2)

Here is a Python program to create Fibonacci series up to n using Lambda:

```python
from functools import reduce

fib_series = lambda n: reduce(lambda x, _: x+[x[-1]+x[-2]],
                                range(n-2), [0, 1])

print("Fibonacci series upto 2:")
print(fib_series(2))
print("\nFibonacci series upto 5:")
print(fib_series(5))
print("\nFibonacci series upto 6:")
print(fib_series(6))
print("\nFibonacci series upto 9:")
print(fib_series(9))
```

This program uses the `reduce` function from the `functools` module to generate the Fibonacci series. The lambda function `fib_series` takes an input `n` and uses `reduce` to iterate over a range of `n-2` elements. It starts with an initial list `[0, 1]` and for each iteration, it appends the sum of the last two elements to the list. The resulting list is the Fibonacci series up to `n`. The program then prints the Fibonacci series for different values of `n`.


In [ ]:
# Evaluating response
from functools import reduce
fib_series = lambda n: reduce(lambda x, _: x+[x[-1]+x[-2]], range(n-2), [0,1])

print("Fibonacci series upto 2:")
print(fib_series(2))
print("\nFibonacci series upto 5:")
print(fib_series(5))
print("\nFibonacci series upto 6:")
print(fib_series(6))
print("\nFibonacci series upto 9:")
print(fib_series(9))

Fibonacci series upto 2:
[0, 1]

Fibonacci series upto 5:
[0, 1, 1, 2, 3]

Fibonacci series upto 6:
[0, 1, 1, 2, 3, 5]

Fibonacci series upto 9:
[0, 1, 1, 2, 3, 5, 8, 13, 21]


In [ ]:
genai_app(prompt_3)

Python is a high-level programming language that is known for its simplicity and readability. It was created by Guido van Rossum and first released in 1991. Python emphasizes code readability and uses indentation and whitespace to define code blocks, making it easy to understand and write. It supports multiple programming paradigms, including procedural, object-oriented, and functional programming. Python has a large standard library and a vast ecosystem of third-party libraries, making it suitable for a wide range of applications, including web development, data analysis, artificial intelligence, and scientific computing. Python is widely used and has a strong community of developers who contribute to its growth and development.


In [ ]:
genai_app("")